In [ ]:
import numpy as np
import math

### Вычислительные эксперименты, иллюстрирующие явление концентрации равномерной меры на поверхности евклидовой сферы в малой окрестности экватора

<i>Статья:</i>
<br/>
<i>Горбунов Э.А.</i>
<br/>
<i>МФТИ (Москва, Россия),</i>
<br/>
<i>Воронцова Е.А.</i>
<br/>
<i>ДВФУ (Владивосток, Россия),</i>
<br/>
<br/>
<i>arxiv (in process)</i>

In [ ]:
def montecarlo(f, X, q, n, eps, gr = False, fs = False, s = []):
    """
    Function to compute the Monte Carlo estimate of the expectation
    E[f(X)], with N samples
   
    Inputs:
    f - a Python function that applies a chosen mathematical function to
    each entry in a numpy array
    X - a Python function that takes N as input and returns
    independent individually distributed random samples from a chosen
    probability distribution
    N - the number of samples to use
    q -- additional parametr for f
    """
    N = 2
    if fs:
        Fx = [f(X(n), q, n, s)]
    else:    
        Fx = [f(X(n), q, n)]
        
    mc_err = eps
    mc_int = 1
    
    if gr:
        plt_estint = []
        plt_esterr = []
        
    while mc_err >= eps or N < 10:
        
        if fs:
            Fx.append(f(X(n), q, n, s))
        else:    
            Fx.append(f(X(n), q, n))
        
        int_sum = np.sum(Fx)
        mc_int = int_sum / N
        
        sq = [x*x for x in Fx]
        sq = abs((np.sum(sq)/N - mc_int*mc_int) / N)
        mc_err = math.sqrt(sq)
        
        if gr:
            plt_estint.append(mc_int)
            plt_esterr.append(mc_err) 
        
        N += 1

    if gr:    
        
        %matplotlib notebook
        import matplotlib.pyplot as plt
        plt.grid(True)
        plt.plot(range(2, N), plt_estint, label = 'mc_int')
        twope = [2*pe for pe in plt_esterr]
        mtwope = [-2*pe for pe in plt_esterr]
        plt.plot(range(2, N), [sum(pe) for pe in zip(plt_estint, twope)])
        plt.plot(range(2, N), [sum(pe) for pe in zip(plt_estint, mtwope)])
        plt.legend()        

    return mc_int, N-1

In [ ]:
def func(e, q, n):
    # Inputs:  
    # e - a numpy array of values
    # q - norm
    # n - length of e
    norm_e = np.linalg.norm(e, q)
    sq = norm_e*norm_e
    if q == np.inf:
        return sq  / pow(n, -1.0)
    else:    
        return sq  / pow(n, 2./q - 1)

In [ ]:
# Generating uniformly distributed numbers on a sphere
def get_rand_vec(dims):
    e = np.random.standard_normal(int(dims))
    return e / np.linalg.norm(e)

In [ ]:
nrange = range(10, int(1e5), 10)
narr = np.array(nrange)

plt1 = []
q = np.inf
for i, n in enumerate(narr):
    
    pract_estimate, N = montecarlo(func, get_rand_vec, q, n, 1e-1, False) 
    if q == np.inf:
        theor_estimate = 16*math.log(n) - 8 #* n2q
    else:
        theor_estimate = min([q - 1, 16*math.log(n) - 8]) #* n2q
    plt1.append(pract_estimate / theor_estimate)
    #print('n = ', n, ', pract  = ', pract_estimate, ', from lemma = ', theor_estimate, N) 
        


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.grid(True)
plt.plot(nrange, plt1)
plt.xlabel(r"$n$", fontsize=12)
plt.ylabel(r"$C_p/C_t$", fontsize=10)
plt.title(r"$q = \infty$", fontsize = 11);
#plt.legend()

#### Второе неравенство, с вектором $s$

In [ ]:
def func_es(e, q, n, s):
    # Inputs:  
    # e - a numpy array of values
    # q - norm
    # n - length of e
    # s - vector
    norm_e = np.linalg.norm(e, q)
    sq_e = norm_e*norm_e
    
    se = s.T.dot(e)
    se2 = se*se
    
    if q == np.inf:
        return float(sq_e*se2  / pow(n, -2.0)) 
    else:    
        return float(sq_e*se2  / pow(n, 2./q - 2)) 

Не ограничивая общности, будем считать, что $s$ --- просто орт $(1, \, 0, \, 0, \, \ldots)$

In [ ]:
nrange = range(10, int(1e5), 100)
narr = np.array(nrange)

plt2 = []
q = np.inf
for i, n in enumerate(narr):
    
    s = np.zeros([n, 1])
    s[0] = 1
    
    pract_estimate, N = montecarlo(func_es, get_rand_vec, q, n, 1e-1, False, True, s) 
    
    norm_s = np.linalg.norm(s, 2)
    sq_norm_s = norm_s*norm_s
    
    if q == np.inf:
        theor_estimate = math.sqrt(3)*sq_norm_s*(32*math.log(n) - 8) 
    else:
        theor_estimate = math.sqrt(3) * sq_norm_s*min([2*q - 1, 32*math.log(n) - 8]) 
    plt2.append(pract_estimate / theor_estimate)
    print('n = ', n, ', pract  = ', pract_estimate, ', from lemma = ', theor_estimate, N) 
        


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.grid(True)
plt.plot(nrange, plt2)
plt.xlabel(r"$n$", fontsize=12)
plt.ticklabel_format(style='sci', axis='y', scilimits=(-2, 2))
plt.ylabel(r"$C_p/C_t$", fontsize=11)
plt.title(r"$q = \infty$", fontsize = 12);